In [112]:
import utils
import numpy as np
import os

In [114]:
players_info_list = utils.get_dataframes_csv_file('../scraping/csv/players_infos')
players_stats_list = utils.get_dataframes_csv_file('../scraping/csv/players_stats')

In [115]:
schedule_df_list = utils.get_dataframes_csv_file('../scraping/csv/schedule' )
standing_df_list = utils.get_dataframes_csv_file('../scraping/csv/standing')
team_stats_df_list = utils.get_dataframes_csv_file('../scraping/csv/team_stats')
team_stats_opponent_df_list = utils.get_dataframes_csv_file('../scraping/csv/team_stats_opponent')
team_stats_differential_df_list = utils.get_dataframes_csv_file('../scraping/csv/team_stats_differential')

# Players Info

Convertir le poid et la taille en kilogramme et en mètre

In [3]:
def convert_to_meters(ht : str):
    if ht == '' or ht == 0:
        return None

    feet, inches = ht.split('-')
    if(inches == ''):
        if(feet == ''):
            return None
        inches = 0

    total_inches = int(feet) * 12 + int(inches)
    meters = total_inches * 0.0254
    return meters

def convert_to_kg(weight_lbs: float):
    if isinstance(weight_lbs, str):
        if weight_lbs == '-' or weight_lbs == '':
            return None
        else:
            weight_lbs = float(weight_lbs)

    if weight_lbs <= 0:
        return None

    weight_kg = weight_lbs * 0.45359237
    return weight_kg

for df in players_info_list:
    df['HT'] = df['HT'].apply(convert_to_meters)
    df['WT'] = df['WT'].apply(convert_to_kg)

Supprimer les colonnes Pre-Draft Team et Draft Status

In [4]:
for i,df in enumerate(players_info_list):
    players_info_list[i]=players_info_list[i].drop('Pre-Draft Team', axis=1)
    players_info_list[i]=players_info_list[i].drop('Draft Status', axis=1)

In [5]:
utils.dflist_to_csv(players_info_list, 'players_infos')

# Players Stats

In [288]:
import numbers

def is_numeric(value):
    return isinstance(value, numbers.Number)

# Convertir les valeurs numérique qui sont représenté en str en float
for df in players_stats_list:
    numeric_cols = ['GP', 'MIN', 'PTS', 'FGM', 'FGA', 'FG%', '3PM', '3PA', '3P%', 'FTM', 'FTA', 'FT%', 'ORB', 'DRB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF']
    
    for col in numeric_cols:
        try:
            df[col] = df[col].apply(lambda x: float(x.replace(",", ".")) if not is_numeric(x) and not x == '-' else x)
        except ValueError as e: 
            df[col] = df[col].apply(lambda x: (x[:-1].replace(".", '').replace(",", ".") + '.' + str(int(x.split('.')[-1]))))

In [290]:
utils.dflist_to_csv(players_stats_list, 'players_stats')

# Schedules

In [26]:
schedule_df_list[0]

,DATE,OPPONENT,RESULT,W-L,Hi Points,Hi Rebounds,Hi Assists,Home_Away,Title
0,"Wed, Oct 29",Sacramento,W95-77,1-0,Curry 24,Curry 10,Thompson 6,H,Golden State Warriors 2014-15 Regular Season N...
1,"Sat, Nov 1",Los Angeles,W127-104,2-0,Thompson 41,Bogut 10,Curry 10,A,Golden State Warriors 2014-15 Regular Season N...
2,"Sun, Nov 2",Portland,W95-90,3-0,Thompson 29,Bogut 12,Curry 6,H,Golden State Warriors 2014-15 Regular Season N...
3,"Wed, Nov 5",LA,W121-104,4-0,Curry 28,Bogut 14,Curry 7,A,Golden State Warriors 2014-15 Regular Season N...
4,"Sat, Nov 8",Houston,W98-87,5-0,Curry 34,Bogut 18,Thompson 6,H,Golden State Warriors 2014-15 Regular Season N...
...,...,...,...,...,...,...,...,...,...
77,"Tue, Apr 7",New Orleans,L103-100,63-15,Curry 25,Green 14,Curry 9,H,Golden State Warriors 2014-15 Regular Season N...
78,"Thu, Apr 9",Portland,W116-105,64-15,Curry 45,Green 14,Curry 10,A,Golden State Warriors 2014-15 Regular Season N...
79,"Sat, Apr 11",Minnesota,W110-101,65-15,Curry 34,Green 14,Curry 7,A,Golden State Warriors 2014-15 Regular Season N...
80,"Mon, Apr 13",Memphis,W111-107,66-15,Thompson 42,Green 9,Curry 8,A,Golden State Warriors 2014-15 Regular Season N...


In [43]:
import datetime

def convert_date_format(date_str):
    """
    Convertit une date au format "Jour de la semaine, Mois Jour" en format "DD/MM/YY".
    """
    try:
        date = datetime.datetime.strptime(date_str, '%a, %b %d')
        return date.strftime(f'{date.day}/{date.month}')
    except ValueError as e:
        if date_str[5] == 'F': # Gérer le cas du 29 Février
            return '29/02'
        else:
            print(f"Error converting date: '{date_str}'. Error: {e}")
            return date_str

def get_player_name(player_str):
    """
    player_str : "Name Number" ; return Name.
    """
    if not player_str or not isinstance(player_str, str):
        return None
    parts = player_str.split(' ')
    player_name = parts[0]
    return player_name

In [66]:
# Supprimer les lignes où la colonne 'DATE' contient la valeur 'DATE' (erreur dans les données)
for i,df in enumerate(schedule_df_list):
    schedule_df_list[i] = df[df['DATE'] != 'DATE']

In [67]:
for df in schedule_df_list:
    # Conversion de la date
    df['DATE'] = df['DATE'].apply(convert_date_format)

    # Création de la colonne W_L
    df['WorL'] = df['RESULT'].str[0]

    if 'W-L' in df.columns:
        df[['WIN', 'LOSE']] = df['W-L'].str.extract(r'(\d+)-(\d+)', expand=True).astype(int)
    else:
        df['WIN'] = None
        df['LOSE'] = None
        df['W-L'] = None


    # Création des colonnes PTS et PTSO
    df[['PTS', 'PTSO']] = df['RESULT'].str.extract(r'(\d+)-(\d+)', expand=True).astype(int)

    # Extraction du nom du joueur
    df['Hi Points'] = df['Hi Points'].apply(get_player_name)
    df['Hi Rebounds'] = df['Hi Rebounds'].apply(get_player_name)
    df['Hi Assists'] = df['Hi Assists'].apply(get_player_name)

In [69]:
for i,df in enumerate(schedule_df_list):
    schedule_df_list[i]=schedule_df_list[i].drop('W-L', axis=1)
    schedule_df_list[i]=schedule_df_list[i].drop('RESULT', axis=1)

In [71]:
utils.dflist_to_csv(schedule_df_list, 'schedules')

# Standings

In [98]:
for df in standing_df_list:

    df[['HOME-W','HOME-L']] = df['HOME'].str.extract(r'(\d+)-(\d+)', expand=True).astype(int)
    df[['AWAY-W','AWAY-L']] = df['AWAY'].str.extract(r'(\d+)-(\d+)', expand=True).astype(int)
    df[['DIV-W','DIV-L']] = df['DIV'].str.extract(r'(\d+)-(\d+)', expand=True).astype(int)
    df[['CONF-W','CONF-L']] = df['CONF'].str.extract(r'(\d+)-(\d+)', expand=True).astype(int)
    df[['Last10-W','Last10-L']] = df['L10'].str.extract(r'(\d+)-(\d+)', expand=True).astype(int)

In [99]:
for df in standing_df_list:
    df['GB'] = df['GB'].str.replace('-', '0').astype(float)

In [100]:
for i,df in enumerate(standing_df_list):
    standing_df_list[i]=standing_df_list[i].drop('HOME', axis=1)
    standing_df_list[i]=standing_df_list[i].drop('AWAY', axis=1)
    standing_df_list[i]=standing_df_list[i].drop('DIV', axis=1)
    standing_df_list[i]=standing_df_list[i].drop('CONF', axis=1)
    standing_df_list[i]=standing_df_list[i].drop('L10', axis=1)

In [103]:
utils.dflist_to_csv(standing_df_list, 'standings')

# Teams stats

In [110]:
utils.dflist_to_csv(team_stats_df_list, 'teams_stats')
utils.dflist_to_csv(team_stats_differential_df_list, 'teams_stats_differential')
utils.dflist_to_csv(team_stats_opponent_df_list, 'teams_stats_opponent')